In [58]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from lightgbm import LGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [83]:
train = pd.read_csv('../dataset/del_outlier_clustering_train.csv', index_col = 0)
test = pd.read_csv('../dataset/test.csv', index_col = 0)
sample_submission = pd.read_csv('../dataset/sample_submission.csv', index_col = 0)

In [84]:
column_number = {}

for i, column in enumerate(sample_submission.columns):
    column_number[column] = i

def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [85]:
"""
bins= [0, 100, 200, 300, 400, 500, 600, 700, 800, 900 ,1000]
labels = ['0', '1','2','3','4','5','6','7','8','9']

train['fiberID1_cut'] = pd.cut(train['fiberID'], bins= bins, labels = labels)
test['fiberID1_cut'] = pd.cut(test['fiberID'], bins= bins, labels = labels)

"""

"\nbins= [0, 100, 200, 300, 400, 500, 600, 700, 800, 900 ,1000]\nlabels = ['0', '1','2','3','4','5','6','7','8','9']\n\ntrain['fiberID1_cut'] = pd.cut(train['fiberID'], bins= bins, labels = labels)\ntest['fiberID1_cut'] = pd.cut(test['fiberID'], bins= bins, labels = labels)\n\n"

In [86]:
train_x = train.drop(columns = ['type', 'type_num'], axis = 1)
train_y = train['type_num']

test_x = test

In [87]:
train_x['fiberID2'] = 0
train_x.loc[train_x['fiberID'] > 640, 'fiberID2'] = 1

test_x['fiberID2'] = 0
test_x.loc[test_x['fiberID'] > 640, 'fiberID2'] = 1

train_x['fiberID2'] = train_x['fiberID2'].astype('category')
test_x['fiberID2'] = test_x['fiberID2'].astype('category')

In [88]:
train_x.drop(columns = 'fiberID', inplace = True)
test_x.drop(columns = 'fiberID', inplace = True)

In [91]:
train_x.head()

,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,...,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,psf_cluster,fiber_cluster,model_cluster,fiberID2
id,,,,,,,,,,,,,,,,,,,,,
0,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,...,20.422446,22.749241,21.465534,21.364187,21.020605,21.147340,3,2,3,0
1,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,...,20.646660,21.492955,20.758527,20.753925,20.693389,20.512314,3,2,3,1
2,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,...,16.808006,17.818063,16.697434,16.641249,16.660177,16.688928,0,4,1,0
3,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,...,19.455117,20.770711,20.001699,19.889798,19.758113,19.552855,2,0,0,1
4,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,...,18.376141,24.877052,23.147993,21.475342,19.487330,18.375655,1,1,2,0


In [93]:
lgb_params = {'learning_rate':0.03,
              'max_depth':6,
              'boosting':'gbdt',
              'objective':'multiclass',
              'metric':'multi_logloss',
              'num_leaves':31,
              'num_class':19}

lgb_train = lgb.Dataset(train_x, train_y)

cv_output = lgb.cv(lgb_params,
                   lgb_train,
                   num_boost_round = 5000,
                   categorical_feature = ['psf_cluster','fiber_cluster','model_cluster','fiberID2'],
                   nfold = 5,
                   early_stopping_rounds = 75,
                   verbose_eval = 10,
                   show_stdv = False)

best_rounds = np.array(list(cv_output.values())).shape[1]
print('best_rounds :', best_rounds)

C:\Users\PARK\Anaconda3\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['fiberID2', 'fiber_cluster', 'model_cluster', 'psf_cluster']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\PARK\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[10]	cv_agg's multi_logloss: 1.55672
[20]	cv_agg's multi_logloss: 1.24018
[30]	cv_agg's multi_logloss: 1.03433
[40]	cv_agg's multi_logloss: 0.89334
[50]	cv_agg's multi_logloss: 0.802689
[60]	cv_agg's multi_logloss: 0.74837
[70]	cv_agg's multi_logloss: 0.658192
[80]	cv_agg's multi_logloss: 0.634073
[90]	cv_agg's multi_logloss: 0.590438
[100]	cv_agg's multi_logloss: 0.557176
[110]	cv_agg's multi_logloss: 0.540911
[120]	cv_agg's multi_logloss: 0.526876
[130]	cv_agg's multi_logloss: 0.512835
[140]	cv_agg's multi_logloss: 0.500849
[150]	cv_agg's multi_logloss: 0.490484
[160]	cv_agg's multi_logloss: 0.482249
[170]	cv_agg's multi_logloss: 0.473835
[180]	cv_agg's multi_logloss: 0.467613
[190]	cv_agg's multi_logloss: 0.465279
[200]	cv_agg's multi_logloss: 0.461618
[210]	cv_agg's multi_logloss: 0.457449
[220]	cv_agg's multi_logloss: 0.455534
[230]	cv_agg's multi_logloss: 0.451416
[240]	cv_agg's multi_logloss: 0.470015
[250]	cv_agg's multi_logloss: 0.453347
[260]	cv_agg's multi_logloss: 0.454797


In [94]:
lgb_model = lgb.train(lgb_params,
                      lgb_train,
                      categorical_feature = ['psf_cluster','fiber_cluster','model_cluster','fiberID2'],
                      num_boost_round = best_rounds)

In [ ]:
pred = lgb_model.predict(test_x)

In [ ]:
submission = pd.DataFrame(data = pred, columns = sample_submission.columns, index = sample_submission.index)

In [ ]:
qso_idx = test_x.loc[(test_x['fiberID2'] == '7') | (test_x['fiberID2'] == '8') | (test_x['fiberID2'] == '9')]

submission.loc[qso_idx.index] = np.nan
submission.loc[qso_idx.index, 'QSO'] = 1

submission.loc[qso_idx.index, submission.drop(columns = 'QSO').columns] = 0

In [ ]:
submission.to_csv('../submission/abc1817.csv')

In [ ]:
submission.head()